Set system path so that pendulum module can be found.

In [1]:
import sys
sys.path.insert(1, '../')
print(sys.executable)
print(sys.version)

%matplotlib ipympl

/home/ms/prg/pendulum/venv/bin/python3
3.8.8 (default, Apr 13 2021, 19:58:26) 
[GCC 7.3.0]


Import packages

In [2]:
import numpy as np
from pendulum import controller, pendulum, sim, utils
from pendulum.viz import Visualizer
from IPython.display import HTML
import matplotlib.pyplot as plt

We set the timestep of the simulation (`dt`), the total simulation time (`t_final`), and create a pendulum. We set the pendulum to be in the upright position: 

$$ [x, \dot{x}, \theta, \dot{\theta}] = [0,0,0,0]$$

In [3]:
dt = 0.01
t_final = 15
pend = pendulum.Pendulum(2.0, 3.0, 2.0, initial_state=np.array([0,0,-.05,0]))

These parameters define our force function. We can define this function however we like; here, it is a sinusoidal impulse function that peaks at 4 seconds.

In [10]:
fig, ax = plt.subplots()
c1, c2, c3, c4 = 2, 3.0, 7, 2
fshift = 6
force_fn = lambda t: c1 * np.sin(c2*t) * c3/(c4*np.sqrt(np.pi)) * np.exp(-((t-fshift)/c4)**2)
fx = np.linspace(0,t_final, 500)
fy = force_fn(fx)
ax.plot(fx, fy)

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

Define the simulation with our parameters

In [11]:
simulation = sim.Simulation(dt, t_final, force_fn)

We create a new LQR controller. $Q$ has the weighted cost function over the inputs; here, we penalize error in $\theta$ highest. We also want to penalize control input, becuase it's assumed that we have finite actuation.

In [44]:
kp = 60
ki = 0
kd = 10

cont = controller.PID(kp,ki,kd)

`simulate` actually runs the simulation. It calculates the pendulum state for each dt from t=0 to t=`t_final`.

In [45]:
results = simulation.simulate(pend, cont, plot=False)

100%|██████████| 1501/1501 [00:00<00:00, 2596.67it/s]


We can create plots on the `results` dataframe returned by the simulation. For example, here is a plot of the states:

In [46]:
fig, ax = plt.subplots()
for s in results['state']:
    ax.plot(results[('state', s)], label=s)
    ax.legend()

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

Here is a plot of the control actions over time.

In [47]:
fig, ax = plt.subplots()
ax.plot(results['control action'], label='control action')
ax.plot(results['forces'], label='force')
ax.plot(results[('state', 't')], label='theta')
ax.legend()

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

We can create an animation of our results using the `Visualizer` class:

In [48]:
visualizer = Visualizer(results, pend, speed=4)
anim = visualizer.animate((9.5, 5))

../pendulum/viz.py:83: RuntimeWarning: More than 20 figures have been opened. Figures created through the pyplot interface (`matplotlib.pyplot.figure`) are retained until explicitly closed and may consume too much memory. (To control this warning, see the rcParam `figure.max_open_warning`).
  fig, ax = plt.subplots(figsize=size)


Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

Some systems may have trouble displaying animations in the browser. We can prerender the animation as an HTML5 video which will display at full speed on all platforms (this cell may take a while to execute.) 

In [ ]:
HTML(anim.to_html5_video())